In [22]:
import joblib
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [23]:
# Загружаем модели
classifier = joblib.load('../src/models/classifier_lgbm.joblib')
regressor = joblib.load('../src/models/rf_regressor.joblib')

threshold = 0.51
mae = 363268  # MAE из регрессии

In [60]:
def evaluate_and_predict(startups):
    """
    startups: DataFrame с колонками:
        ['startup_stage', 'industry', 'region', 'requested_amount',
        'pre_money_valuation', 'team_size', 'founders_experience_years',
        'annual_revenue']
    """
    results = []

    # 1. Принятие решения по финансированию
    reject_proba = classifier.predict_proba(startups)[:, 1]
    
    for i, proba in enumerate(reject_proba):
        decision = 'ОТКАЗАНО' if proba >= threshold else 'ПРИНЯТО'
        proba_percent = proba * 100

        result = {
            'ID': i,
            'Вероятность отказа': f'{int(proba_percent)}%',
            'Решение': decision,
            'Минимальные инвестиции': 'ОТКАЗ',
            'Оптимальные инвестиции': 'ОТКАЗ',
            'Максимальные инвестиции': 'ОТКАЗ'
        }

        # 2. Регрессия (только если принято)
        if decision == 'ПРИНЯТО':
            # Подготовка (логарифмирование) признаков для регрессии.
            startup = pd.DataFrame(startups.iloc[[i]]).copy()
            for feature in ['requested_amount', 'pre_money_valuation', 'team_size',
                            'founders_experience_years', 'annual_revenue']:
                startup[f'{feature}_log'] = np.log1p(startup[feature].astype(float))
            
            log_pred = regressor.predict(startup)[0]
            real_pred = int(np.expm1(log_pred))
            
            result['Оптимальные инвестиции'] = real_pred
            result['Минимальные инвестиции'] = real_pred - mae
            result['Максимальные инвестиции'] = real_pred + mae

        results.append(result)

    # Результаты.
    return pd.DataFrame(results)

In [61]:
startups = pd.DataFrame({
    'startup_stage': ['Pre-Seed', 'Seed', 'Series A', 'Idea', 'Series B', 'Seed'],
    'industry': ['ClimateTech', 'FinTech', 'HealthTech', 'E-commerce', 'AI', 'EdTech'],
    'region': ['US', 'EU', 'US', 'LATAM', 'US', 'APAC'],
    'requested_amount': [1_200_000, 2_500_000, 6_000_000, 600_000, 15_000_000, 1_800_000],
    'pre_money_valuation': [6_500_000, 12_000_000, 28_000_000, 3_500_000, 55_000_000, 9_000_000],
    'team_size': [7, 10, 14, 3, 18, 6],
    'founders_experience_years': [2, 5, 9, 1, 12, 4],
    'annual_revenue': [0, 120_000, 950_000, 0, 2_500_000, 80_000]
})

startups

,startup_stage,industry,region,requested_amount,pre_money_valuation,team_size,founders_experience_years,annual_revenue
0,Pre-Seed,ClimateTech,US,1200000,6500000,7,2,0
1,Seed,FinTech,EU,2500000,12000000,10,5,120000
2,Series A,HealthTech,US,6000000,28000000,14,9,950000
3,Idea,E-commerce,LATAM,600000,3500000,3,1,0
4,Series B,AI,US,15000000,55000000,18,12,2500000
5,Seed,EdTech,APAC,1800000,9000000,6,4,80000


In [62]:
df_results = evaluate_and_predict(startups)
df_results

,ID,Вероятность отказа,Решение,Минимальные инвестиции,Оптимальные инвестиции,Максимальные инвестиции
0,0,64%,ОТКАЗАНО,ОТКАЗ,ОТКАЗ,ОТКАЗ
1,1,68%,ОТКАЗАНО,ОТКАЗ,ОТКАЗ,ОТКАЗ
2,2,10%,ПРИНЯТО,5326192,5689460,6052728
3,3,93%,ОТКАЗАНО,ОТКАЗ,ОТКАЗ,ОТКАЗ
4,4,11%,ПРИНЯТО,13859663,14222931,14586199
5,5,52%,ОТКАЗАНО,ОТКАЗ,ОТКАЗ,ОТКАЗ
